In [1]:
import pandas as pd
import numpy as np
from scipy.constants import physical_constants
import matplotlib.pyplot as plt
plt.style.use('..\JGW.mplstyle')

# Notebook version only
import os
os.chdir(r'C:\Users\Administrator\Documents\GitHub\DataHandlers\notebooks')
# End Notebook version only

from ASTM_E698_2011 import PeakTempCorrection, iter_refine, get_Z, get_k
from LinReg import PolyReg

In [2]:
# --------------------------------------------------------------------------------------------
# USER DEFINED PARAMETERS
raw = pd.read_csv(r'Indium-VR_JGW-A-43.csv')
mass = 5.491 #in mg
Therm_Resist = 0.49441 #in K/mW

beta_choose = 10
T_Arrhenius = 200
tolerance_frac = 0.005
# --------------------------------------------------------------------------------------------

In [3]:
#Import and fit
df = PeakTempCorrection(raw, Therm_Resist, mass)

In [6]:
Rate_Corr = PolyReg(df['Heat Rate'], df['Lag Corr. ΔT'], 1)
# Rate_Corr.report()

R = physical_constants['molar gas constant'][0]
logHeatRate_vs_Tinv = PolyReg(1/df['Lag Corr. Temp (K)'], df['log10(Heat Rate)'], 1)
Ea = -2.19 * R * logHeatRate_vs_Tinv.coef[0]

# Refine Ea
# T_chosen is the Corr. Peak Temp (K) at beta_choose
T_chosen = df.loc[df['Heat Rate'] == beta_choose, 'Lag Corr. Temp (K)']
ref_Ea = iter_refine(Ea,
                     logHeatRate_vs_Tinv.coef[0],
                     T_chosen,
                     tolerance_frac)
Z = get_Z(ref_Ea, T_chosen, beta_choose)
k = get_k(ref_Ea, Z, T_Arrhenius)

report = pd.Series({'Ea': ref_Ea, 'Z': Z, 'k': k})
print(report)

df

0      1097539.083958
1    1146532.23298521
2    1146846.79536239
Name: Ea (J/mol), dtype: object
Ea         1146846.79536239
Z     3.00676372637068e+140
k     9.08447992449615e-160
dtype: object


,Heat Rate,Peak Temp (C),Peak Height (W/g),log10(Heat Rate),Height (mW),Lag Corr. Temp (C),Lag Corr. Temp (K),Lag Corr. ΔT
0,1,155.40,1.3566,0.000000,0.007449,155.396317,428.546317,-0.554243
1,2,155.50,2.4256,0.301030,0.013319,155.493415,428.643415,-0.457145
2,5,155.71,4.6811,0.698970,0.025704,155.697292,428.847292,-0.253268
3,10,155.97,7.1608,1.000000,0.039320,155.950560,429.100560,0.000000
4,15,156.17,8.7492,1.176091,0.048042,156.146248,429.296248,0.195688
5,20,156.37,9.8964,1.301030,0.054341,156.343133,429.493133,0.392573
6,50,157.38,13.2385,1.698970,0.072693,157.344060,430.494060,1.393500
7,100,158.87,15.1974,2.000000,0.083449,158.828742,431.978742,2.878182
8,150,160.25,16.0861,2.176091,0.088329,160.206329,433.356329,4.255770
9,200,161.58,16.7268,2.301030,0.091847,161.534590,434.684590,5.584030


In [8]:
j = PolyReg(df['Lag Corr. Temp (K)'], df['Heat Rate'], 1)
j.report()

coef_deg1          32.616329
std_err_deg1        0.837009
coef_deg0      -13984.205373
std_err_deg0      360.289465
r_squared           0.994759
s_y                 5.439996
dtype: float64